# Imports

In [1]:
from hw4lib.data import (
    H4Tokenizer,
    ASRDataset,
    verify_dataloader
)

# Implementations
- `NOTE`: All of these implementations have detailed specification, implementation details, and hints in their respective source files. Make sure to read all of them in their entirety to understand the implementation details!

## Dataset Implementation
- `TODO`: Implement the `ASRDataset` class in `hw4lib/data/asr_dataset.py`. 
- You will have to implement parts of `__init__` and completely implement the `__len__`, `__getitem__` and `collate_fn` methods. 
- `TODO`: Then run the cell below to check your implementation.


In [2]:
!python -m tests.test_dataset_asr


Loading data for train-clean-100 partition...
100%|██████████████████████████████████████████| 28/28 [00:00<00:00, 463.41it/s]
Loading data for test-clean partition...
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1362.23it/s]

Running tests for category: ASRDataset Train
--------------------------------------------------------------------------------

[01/01]    Running:  Test a Train instance of ASRDataset class
Testing __init__ method ...
Test Passed: Dataset length matches FBANK files.
Test Passed: Dataset length matches TRANSCRIPT files.
Test Passed: Order alignment between FBANK files and TRANSCRIPT files is correct.
Test Passed: Alignment between features and transcripts is correct.
Test Passed: All features have the correct number of dimensions (num_feats).
Test Passed: All transcripts are decoded correctly after removing SOS and EOS tokens.
Testing __getitem__ method ...
Test Passed: All samples have correct feature dimensions and transcript alignment.
Te

## Model Implementations

- `TODO`: Implement the `CrossAttentionLayer` class in `hw4lib/model/sublayers.py`.
- `TODO`: Implement the `CrossAttentionDecoderLayer` class in `hw4lib/model/decoder_layers.py`.
- `TODO`: Implement the `SelfAttentionEncoderLayer` class in `hw4lib/model/encoder_layers.py`. This will be mostly a copy-paste of the `SelfAttentionDecoderLayer` class in `hw4lib/model/decoder_layers.py` with one minor diffrence: it can attend to all positions in the input sequence.
- `TODO`: Implement the `EncoderDecoderTransformer` class in `hw4lib/model/transformers.py`.

### Transformer Sublayers
- `TODO`: Now, Implement the `CrossAttentionLayer` class in `hw4lib/model/sublayers.py`.
- `NOTE`: You should have already implemented the `SelfAttentionLayer`, and `FeedForwardLayer` classes in `hw4lib/model/sublayers.py`.
- `TODO`: Then run the cell below to check your implementation.

In [3]:
!python -m tests.test_sublayer_crossattention


Running tests for category: CrossAttentionLayer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the cross-attention sublayer
Testing initialization ...
Test Passed: All layers exist and are instantiated correctly
Testing forward shapes ...
Test Passed: Forward pass returns the correct shapes
Testing padding mask behaviour ...
Test Passed: Padding mask is applied correctly
Testing cross-attention behaviour ...
Test Passed: Cross-attention behavior is correct
Testing residual connection ...
Test Passed: Residual connection is applied correctly
[01/01]    PASSED:   Test the cross-attention sublayer


                                  Test Summary                                  
Category:    CrossAttentionLayer           
Results:     1/1 tests passed (100.0%)


### Transformer Cross-Attention Decoder Layer
- `TODO`: Implement the `CrossAttentionDecoderLayer` class in `hw4lib/model/decoder_layers.py`.
- `TODO`: Then run the cell below to check your implementation.


In [4]:
!python -m tests.test_decoderlayer_crossattention


Running tests for category: CrossAttentionDecoderLayer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the cross-attention decoder layer
Testing initialization ...
Test Passed: All sublayers exist and are initialized correctly
Testing forward shapes ...
Test Passed: Forward shapes are as expected
Testing sublayer integration ...
Test Passed: Sublayers interact correctly
Testing cross-attention behavior ...
Test Passed: Cross-attention behaves correctly
[01/01]    PASSED:   Test the cross-attention decoder layer


                                  Test Summary                                  
Category:    CrossAttentionDecoderLayer    
Results:     1/1 tests passed (100.0%)


### Transformer Self-Attention Encoder Layer
- `TODO`: Implement the `SelfAttentionEncoderLayer` class in `hw4lib/model/encoder_layers.py`.
- `TODO`: Then run the cell below to check your implementation.




In [5]:
!python -m tests.test_encoderlayer_selfattention


Running tests for category: SelfAttentionEncoderLayer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the self-attention encoder layer
Testing initialization ...
Test Passed: All sublayers exist and are initialized correctly
Testing forward shapes ...
Test Passed: Forward shapes are as expected
Testing sublayer interaction ...
Test Passed: Sublayers interact correctly
Testing bidirectional attention ...
Test Passed: Bidirectional attention is working correctly
[01/01]    PASSED:   Test the self-attention encoder layer


                                  Test Summary                                  
Category:    SelfAttentionEncoderLayer     
Results:     1/1 tests passed (100.0%)


### Encoder-Decoder Transformer

- `TODO`: Implement the  `EncoderDecoderTransformer` class in `hw4lib/model/transformers.py`.
- `TODO`: Then run the cell below to check your implementation.

In [6]:
!python -m tests.test_transformer_encoder_decoder


Running tests for category: EncoderDecoderTransformer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the encoder-decoder transformer
Testing initialization...
Test Passed: All components initialized correctly
Testing encode method...
Test Passed: Encode method works correctly
Testing decode method...
Test Passed: Decode method works correctly
Testing forward pass...
Test Passed: Forward pass works correctly
Testing encoder-decoder integration...
Test Passed: Encoder-decoder integration works correctly
Testing CTC integration...
Test Passed: CTC integration works correctly
Testing forward propagation order...
Test Passed: Forward propagation order is correct
[01/01]    PASSED:   Test the encoder-decoder transformer


                                  Test Summary                                  
Category:    EncoderDecoderTransformer     
Results:     1/1 tests passed (100.0%)


## Decoding Implementations 
- `TODO`: We highly recommend you to implement the `generate_beam` method of the `SequenceGenerator` class in `hw4lib/decoding/sequence_generator.py`.
- `TODO`: Then run the cell below to check your implementation.
- `NOTE`: This is an optional but highly recommended task for `HW4P2` to ease the journey to high cutoffs!

In [7]:
!python -m tests.test_decoding --mode beam  


Running tests for category: Decoding
--------------------------------------------------------------------------------

[01/01]    Running:  Test beam decoding
Testing Single Batch Beam Search ...
Beam 0  : generated: HELLO WORLD  | expected: HELLO WORLD 
Beam 1  : generated: YELLOW WORLD | expected: YELLOW WORLD
Beam 2  : generated: MELLOW WORLD | expected: MELLOW WORLD
Testing Multi Batch Beam Search ...
Batch 0  : Beam 0  : generated: HELLO WORLD  | expected: HELLO WORLD 
Batch 0  : Beam 1  : generated: YELLOW WORLD | expected: YELLOW WORLD
Batch 0  : Beam 2  : generated: MELLOW WORLD | expected: MELLOW WORLD
Batch 1  : Beam 0  : generated: GOOD BYE     | expected: GOOD BYE    
Batch 1  : Beam 1  : generated: GREAT DAY    | expected: GREAT DAY   
Batch 1  : Beam 2  : generated: GUD NIGHT    | expected: GUD NIGHT   
[01/01]    PASSED:   Test beam decoding


                                  Test Summary                                  
Category:    Decoding                      
Res

# Experiments